# タブーサーチでQAPを解く

QAPでは，互換を解の近傍として使用する．

最良の近傍解を求めるために，$O(n^2)$ 個の近傍解全てに対する目的関数の差分を計算する必要があるが，目的関数の差分を直接計算するのに $O(n)$ の計算量がかかるので，イテレーションごとに $O(n^3)$ の計算量がかかることになる．

これでは計算が重いので，「すべての近傍解に対する目的関数の差分」 $\Delta$ を保持しておくことで計算量を削減する．

1. 初期化として，$\Delta$ を直接計算する: $O(n^3)$
2. 解を更新するたびに，$\Delta$ も更新する: $O(n^2)$

1 は次のように計算される:
置換 $\pi$ の $i_*, j_*$ の値を入れ替えて新しい置換 $\pi'$ が得られるとすると，目的関数の差分は

$$
\sum_{i,j} f_{ij} (d_{\pi'(i)\pi'(j)} - d_{\pi(i)\pi(j)})
= 2 \sum_{i\ne i_*, j_*} (f_{i_* i} - f_{j_* i}) (d_{\pi(i_*)\pi(i)} - d_{\pi(j_*)\pi(i)}).
$$


2 については，例えば次のようにできる: 更新後の解 $\pi'$ の$i_{ ** }, j_{ ** }$ の値を入れ替えた時の目的関数の差分を考える．
- $i_{ ** }, j_{ ** }$ と $i_*, j_*$ に重複がなければ ($O(n^2)$個)，目的関数の差分は $\Delta$ に $i_*, j_*, i_{ ** }, j_{ ** }$ の組み合わせだけの四則演算 ($O(1)$) を足して更新できる
- $i_{ ** }, j_{ ** }$ と $i_*, j_*$ に重複がある場合 ($O(n)$個)，それらに対して1の計算をやり直す ($O(n)$)

以下の実装では，上のやり方とは少し異なる方法で $\Delta$ に相当するデータを保持しているようだ (詳細は不明)

In [1]:
import random
from util import read_qap
Infinity = float('inf')
LOG = False

def mk_rnd_data(n, scale=10):
    f = {}
    d = {}

    for i in range(n):
        f[i,i] = 0
        d[i,i] = 0
    
    for i in range(n-1):
        for j in range(i+1, n):
            f[i,j] = int(random.random() * scale)
            f[j,i] = f[i,j]
            d[i,j] = int(random.random() * scale)
            d[j,i] = d[i,j]

    return n, f, d


def evaluate__(n, f, d, pi):
    cost = 0
    for i in range(n-1):
        for j in range(i+1, n):
            cost += f[i,j] * d[pi[i], pi[j]]
    return cost * 2


def evaluate(n, f, d, pi):
    delta = {}
    for i in range(n):
        for j in range(n):
            delta[i,j] = 0
            for k in range(n):
                delta[i,j] += f[i,k] * d[j, pi[k]]
    cost = 0
    for i in range(n):
        cost += delta[i, pi[i]]
    
    return cost, delta


def construct(n, f, d):
    pi = list(range(n))
    random.shuffle(pi)
    return pi


def diversify(pi):
    n = len(pi)
    missing = set(pi)

    start = int(random.random()*n)
    ind = list(range(n))
    random.shuffle(ind)
    for ii in range(start):
        i = ind[ii]
        pi[i] = missing.pop()

    return pi


def find_move(n, f, d, pi, delta, tabu, iteration):
    """ Returns the next move in tabu search iterations.

    Parameters
    ----------
    n : int
        Size of the problem
    f : dict
        Flow matrix (mapping an index tuple to int)
    d : dict
        Distance matrix (mapping an index tuple to int)
    pi : list
        Incumbent solution (permutation)
    delta : dict
        Auxiliary data for computing objective difference (mapping an index tuple to int)
    tabu : dict
        Tabu list (mapping an index tuple to int (Tabuじゃなくなる時刻)). dict として保持することで計算量O(1)でアクセスできる
    iteration : int
        Current count of iterations

    Returns
    -------
    istar, jstar, minmove*2
        Move of solution ()
    """
    minmove = Infinity
    istar, jstar = None, None
    for i in range(n-1):
        for j in range(i+1, n):
            if tabu[i, pi[j]] > iteration or tabu[j, pi[j]] > iteration:
                continue

            move = delta[j, pi[i]] - delta[j, pi[j]] \
                + delta[i, pi[j]] - delta[i, pi[i]] \
                + 2 * f[i,j] * d[pi[i], pi[j]]

            if move < minmove:
                minmove = move
                istar = i
                jstar = j
    
    if istar != None:
        return istar, jstar, minmove*2
    
    print("blocked, no non-tabu move")
    # clean tabu list
    for i in range(n):
        for j in range(n):
            tabu[i,j] = 0
    return find_move(n, f, d, pi, delta, tabu, iteration)
    

def tabu_search(n, f, d, max_iter, length, report=None):
    """ Perform tabu search for QAP.

    Parameters
    ----------
    n : int
        Size of the problem
    f : dict
        Flow matrix (mapping an index tuple to int)
    d : dict
        Distance matrix (mapping an index tuple to int)
    max_iter : int
        Upper limit for the number of iterations in tabu search
    length : int
        Length of tabu list
    report : Callback, optional
        Callback function (e.g. print) for displaying intermediate results, by default None

    Returns
    -------
    bestsol, bestcost
        Tuple of best solution and best cost
    """
    tabulen = length
    tabu = {}
    for i in range(n):
        for j in range(n):
            tabu[i,j] = 0
    pi = construct(n, f, d)
    cost, delta = evaluate(n, f, d, pi)
    bestcost = cost

    if LOG: print("iteration", 0, "\tcost =", cost, ", best =", bestcost)
    for it in range(max_iter):
        istar, jstar, mindelta = find_move(n, f, d, pi, delta, tabu, it)
        cost += mindelta
        for i in range(n):
            for j in range(n):
                delta[i,j] += (f[i, jstar] - f[i, istar]) * (d[j, pi[istar]] - d[j, pi[jstar]])

        tabu[istar, pi[istar]] = it + tabulen
        tabu[jstar, pi[jstar]] = it + tabulen

        pi[istar], pi[jstar] = pi[jstar], pi[istar]

        if cost < bestcost:
            bestcost = cost
            bestsol = list(pi)
            if report:
                report(bestcost, "it:%d"%it)
        if LOG: print("iteration", it+1, "\tcost =", cost, ", best =", bestcost)
    
    if report:
        report(bestcost, "it:%d"%it)
    
    xcost, xdelta = evaluate(n, f, d, pi)
    assert xcost == cost

    return bestsol, bestcost


In [3]:
folder = '../data/qap/problem/'

n, f, d = read_qap(folder +"tai60a.dat")
print("starting tabu search")
tabulen = n
max_iterations = 30000
pi, cost = tabu_search(n, f, d, max_iterations, tabulen, report=print)

print("final solution: z =", cost)
print(pi)

starting tabu search
8372846 it:0
8299408 it:1
8242376 it:2
8186730 it:3
8134648 it:4
8092252 it:5
8056488 it:6
8011832 it:7
7975890 it:8
7941988 it:9
7906370 it:10
7875094 it:11
7855734 it:12
7838200 it:13
7820776 it:14
7802362 it:15
7782846 it:16
7773942 it:17
7757288 it:18
7742944 it:19
7713204 it:20
7691594 it:21
7674556 it:22
7670070 it:23
7663312 it:24
7652956 it:25
7643346 it:26
7640100 it:27
7635692 it:28
7619012 it:29
7609310 it:30
7603712 it:31
7601486 it:32
7597610 it:33
7590482 it:34
7585448 it:35
7581430 it:36
7575974 it:37
7573144 it:38
7551868 it:39
7529982 it:40
7521292 it:41
7517862 it:42
7517588 it:43
7511084 it:45
7506792 it:46
7502906 it:47
7499506 it:49
7497322 it:50
7497260 it:58
7497062 it:59
7489686 it:72
7488538 it:73
7483670 it:78
7477836 it:79
7466376 it:80
7461196 it:81
7454198 it:82
7451966 it:103
7438938 it:104
7435282 it:124
7430786 it:146
7427158 it:355
7413868 it:357
7411760 it:472
7407686 it:478
7403592 it:483
7401484 it:484
7399952 it:1543
7396702 it:

In [2]:
folder = '../data/qap/problem/'

n, f, d = read_qap(folder +"tai20a.dat")
print("starting tabu search")
tabulen = n
max_iterations = 30000
pi, cost = tabu_search(n, f, d, max_iterations, tabulen, report=print)

print("final solution: z =", cost)
print(pi)

starting tabu search
872774 it:0
840784 it:1
818162 it:2
802742 it:3
789932 it:4
777374 it:5
762298 it:6
759352 it:7
755986 it:8
748588 it:10
737808 it:14
734618 it:15
730112 it:21
724552 it:35
723334 it:36
716898 it:38
716670 it:681
711778 it:682
709448 it:12537
706786 it:13583
703482 it:21866
703482 it:29999
final solution: z = 703482
[9, 8, 11, 19, 18, 2, 13, 5, 16, 10, 4, 6, 14, 15, 17, 1, 3, 7, 12, 0]
